In [1]:
import numpy as np 
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from numba import jit
import json

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [2]:
df = pd.read_csv('../datalets/datalet2.csv',index_col=False)

In [3]:
headers = []

for i in range(df.shape[1]):
    headers.append('period{}'.format(i))
 

df.columns = headers    

display(df)

,period0,period1
0,4.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
3152900,4.0,0.0
3152901,0.0,0.0
3152902,0.0,4.0
3152903,4.0,0.0


In [4]:
counts = {}
for x in range(len(df['period0'].value_counts())):
    counts['{}.0'.format(x)]= df['period0'].value_counts()[x]
    
print(counts)


{'0.0': 1668305, '1.0': 371137, '2.0': 371155, '3.0': 371155, '4.0': 371153}


In [5]:
df['ydata'] = df['period1']
output = pd.pivot_table(data=df, index=['period0'], columns=['period1'], values='ydata', aggfunc='count')
display(output)



period1,0.0,1.0,2.0,3.0,4.0
period0,,,,,
0.0,980391,169422,169466,173339,175687
1.0,183004,65396,48939,39944,33854
2.0,172758,57214,54627,46511,40045
3.0,170582,46075,53460,53602,47436
4.0,161570,33030,44663,57759,74131


In [6]:
#This is a the stochastic markov chain that tells you, if you start at a particular MEV state, where might you end up I am just transposing it for the calculation and it will be transposed back after

tmatrix = output.transpose(copy=False)

#This loop divides by the total counts in each row, to work out the conditional probability of going from one state to another. 

for x in range(len(counts)):
    tmatrix[x] = (tmatrix[x] / counts['{}.0'.format(x)])


transition_matrix = tmatrix.transpose(copy=False)

display(transition_matrix)

period1,0.0,1.0,2.0,3.0,4.0
period0,,,,,
0.0,0.587657,0.101553,0.101580,0.103901,0.105309
1.0,0.493090,0.176204,0.131862,0.107626,0.091217
2.0,0.465461,0.154151,0.147181,0.125314,0.107893
3.0,0.459598,0.124140,0.144037,0.144419,0.127806
4.0,0.435319,0.088993,0.120336,0.155620,0.199732
